<a href="https://colab.research.google.com/github/aravind-chilakamarri/Pyspark/blob/main/Pyspark_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import pyspark
import pandas as pd

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('Practise 6').getOrCreate()

In [5]:
pd_df = pd.read_csv('https://raw.githubusercontent.com/aravind-chilakamarri/Pyspark/main/datasets/test6.csv')
df_pyspark = spark.createDataFrame(pd_df)

In [6]:
df_pyspark.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [7]:
df_pyspark.columns

['Name', 'age', 'Experience', 'Salary']

In [8]:
# we have to create a way where we group all our independent features

# ['age','Experience'] -> treat this group as a new feature (independent feature)

In [10]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [13]:
output = featureassembler.transform(df_pyspark)

In [14]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [15]:
finalized_data=output.select("Independent Features","Salary")

In [16]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [17]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [18]:
# Coefficients
regressor.coefficients

DenseVector([1771.9298, -298.2456])

In [19]:
#Intercepts
regressor.intercept

-21912.280701753818

In [20]:
# Prediction
pred_results=regressor.evaluate(test_data)

In [21]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [29.0,4.0]| 20000|28280.701754385842|
|          [30.0,8.0]| 25000|28859.649122806983|
+--------------------+------+------------------+

